<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/OpenAI_Chat_Completions_16k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI new chat model gpt-3.5-turbo-16k

gpt-3.5-turbo-16k offers 4 times the context length of gpt-3.5-turbo at twice the price: $0.003/1K input tokens and $0.004/1K output tokens.

16k context means the model can now support ~20 pages of text in a single request.

In [31]:
# !pip install tiktoken langchain openai --quiet --upgrade
# !pip install PyPDF2 pymupdf 
# !wget https://uniswap.org/whitepaper-v3.pdf
# !ls -alt whitepaper-v3.pdf


In [8]:
import os,sys
import openai
from dotenv import load_dotenv, find_dotenv
# sys.path.append("../..")

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
print(find_dotenv())
_ = load_dotenv(find_dotenv())
print(os.environ["OPENAI_API_KEY"])

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

from langchain.chains import LLMRequestsChain, LLMChain


llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo')

c:\Users\lenovo\Desktop\LangChainPlayGround\DeeperTutorials\.env
sk-lANo2jIeCWQt94UCCf5d16B7C32744279bF98b06C822D519


d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


[83, 1609, 5963, 374, 2294, 0]

In [11]:
from langchain.document_loaders import PyMuPDFLoader

pages = PyMuPDFLoader('whitepaper-v3.pdf').load()
all_content = ''
for page in pages:
  all_content += page.page_content + "\n"
print(len(all_content))


import tiktoken
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo-16k')
print(encoding)
print(encoding.encode("tiktoken is great!"))

tokens = encoding.encode(all_content)
print(len(tokens))

41715
<Encoding 'cl100k_base'>
[83, 1609, 5963, 374, 2294, 0]
12381


In [12]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k",
  messages = [
    {"role": "system", "content" : "You are a chatbot which can answer questions based on the paper user supplies."},
    {"role": "user", "content" : "I have the following whitepaper of uniswap v3"},
    {"role": "user", "content" : all_content},
    {"role": "user", "content" : "What is liquidity oracle?"}
  ]
)
print(completion)

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 12433 tokens. Please reduce the length of the messages.

In [10]:
completion['choices'][0]['message']['content']

'A liquidity oracle is a mechanism that provides information about the amount of liquidity available in a specific pool or market. It is used to determine the depth and availability of assets within the pool, which helps traders and protocols make informed decisions about trading and pricing.\n\nA liquidity oracle typically reports metrics such as the total liquidity in a pool, the depth of buy and sell orders at different price levels, and the slippage or impact cost for different trading volumes. This information is crucial for traders to determine the best execution strategy and for protocols to optimize their liquidity provision.\n\nLiquidity oracles can be implemented in various ways, including on-chain or off-chain solutions. On-chain liquidity oracles fetch data directly from the blockchain, while off-chain liquidity oracles rely on external data sources or smart contracts to provide liquidity information. These oracles ensure that accurate and up-to-date liquidity data is avail